In [1]:
import os
os.environ["PATH"] = os.environ.get("PATH", "") + ":/opt/ffmpeg-btbn/ffmpeg-*-linux64-gpl/bin"
os.environ["LIBVMAF_MODEL_PATH"] = "/usr/local/share/model"

import subprocess
from IPython.display import display

print(subprocess.check_output(["ffmpeg","-version"], text=True).splitlines()[0])
print("libvmaf" in subprocess.check_output(["ffmpeg","-hide_banner","-filters"], text=True))

ffmpeg version N-121432-g6b961f5963-20251014 Copyright (c) 2000-2025 the FFmpeg developers
True


In [14]:
# Step 1: Imports & Global Config

import os, sys, json, tempfile, pathlib, subprocess
from typing import List, Tuple, Dict, Optional, Any
from concurrent.futures import ThreadPoolExecutor, as_completed

# === Required folders ===
INPUT_DIR  = "/home/indranil/video_process/testbench2"
OUTPUT_DIR = "/home/indranil/video_process/testbench2_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === Search space (you can edit) ===
CRF_LIST = [26, 28, 32, 34]

# === VBV policy (RULE 1 & RULE 2) ===
# By default we DO NOT cap (offline benchmarking).
# If you explicitly set ENABLE_VBV = True, we will add ONLY mild caps:
#   maxrate multipliers: 0.75 and 1.00 of source avg bitrate
#   bufsize              ≈ 2.5 × maxrate
ENABLE_VBV          = False            # RULE 1: default = False
VBV_MAXRATE_MULTS   = [0.75, 1.00]     # RULE 2: mild caps only
VBV_BUFSIZE_FACTOR  = 2.5              # RULE 2: bufsize ~= 2–3 × maxrate

# === Sampling VMAF speed ===
SAMPLING_VMAF_SUBSAMPLE      = 5       # evaluate every 5th frame
SAMPLING_VMAF_DOWNSCALE_HALF = True    # half-res for sampling only (fast)

# === Threads ===
FFMPEG_THREADS = "1"                   # conservative; tune if you have CPU headroom
VMAF_THREADS   = 1

# === Guard/Rescue thresholds (RULE 3) ===
MAX_S_DROP = 0.20    # allow up to 0.20 drop vs sample-best; if larger, take actions described below
MIN_OK_S   = 0.40    # if Final S < 0.40 → unconditional no-cap re-encode at same CRF

# === Sampling clips (two 6s windows) ===
SAMPLE_DURATION     = 6
SAMPLE_OFFSETS_FRAC = [0.2, 0.7]       # early & mid-late

# === Perceptual x265 defaults (moderate) ===
X265_PRESET  = "medium"
X264_PRESET  = "medium"
PERCEPTUAL_X265 = {
    "aq-mode": 2, "aq-strength": 1.0, "psy-rd": 1.0, "psy-rdoq": 1.0, "rd": 4,
    "pools": 1, "frame-threads": 1
}

# === Gentle prefilter (used only if sample-best S < 0.5) ===
GENTLE_PREFILTER = "hqdn3d=1.5:1.5:6:6"

# === Allowed extensions & policy (same extension, same resolution) ===
ALLOWED_EXTS = {".mp4", ".mkv", ".mov", ".avi"}

In [15]:
# Step 2: FFmpeg checks & utilities

def run_cmd(cmd: List[str]) -> Tuple[int, str, str]:
    """Run ffmpeg with fixed-thread setting, capture output."""
    base = ["ffmpeg", "-hide_banner", "-loglevel", "error", "-nostdin", "-threads", FFMPEG_THREADS]
    p = subprocess.Popen(base + cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    out, err = p.communicate()
    return p.returncode, out, err

def check_ffmpeg() -> None:
    r = subprocess.run(["ffmpeg","-hide_banner","-version"], capture_output=True, text=True)
    if r.returncode != 0:
        raise RuntimeError("ffmpeg not found on PATH.")
    enc = subprocess.run(["ffmpeg","-hide_banner","-encoders"], capture_output=True, text=True).stdout
    flt = subprocess.run(["ffmpeg","-hide_banner","-filters"],  capture_output=True, text=True).stdout
    if "libvmaf" not in flt:
        raise RuntimeError("libvmaf filter missing in ffmpeg build.")
    if ("libx265" not in enc) and ("libx264" not in enc):
        raise RuntimeError("libx265/libx264 encoders missing in ffmpeg build.")
    print("FFmpeg OK: libx264/libx265/libvmaf present.")

def ffprobe_json(path: str) -> dict:
    p = subprocess.run(["ffprobe","-v","error","-print_format","json","-show_format","-show_streams", path],
                       capture_output=True, text=True)
    if p.returncode != 0:
        raise RuntimeError(f"ffprobe failed for {path}: {p.stderr}")
    return json.loads(p.stdout)

def get_video_props(path: str) -> dict:
    info    = ffprobe_json(path)
    fmt     = info.get("format", {})
    streams = info.get("streams", [])
    vstreams = [s for s in streams if s.get("codec_type") == "video"]
    if not vstreams:
        raise RuntimeError("No video stream found")
    v = vstreams[0]
    return {
        "duration": float(fmt.get("duration", v.get("duration", 0) or 0)),
        "bit_rate_bps": float(fmt.get("bit_rate", 0)),
        "width": int(v.get("width", 0)),
        "height": int(v.get("height", 0)),
        "pix_fmt": v.get("pix_fmt",""),
        "color_transfer": (v.get("color_transfer") or "").lower(),
        "r_frame_rate": v.get("r_frame_rate") or v.get("avg_frame_rate") or None
    }

def list_input_videos(input_dir: str) -> List[str]:
    p = pathlib.Path(input_dir)
    vids = sorted([str(p/f) for f in os.listdir(p) if (p/f).suffix.lower() in ALLOWED_EXTS])
    return vids

def out_path_for(output_dir: str, src: str, tag: str = "hevc_best") -> str:
    p = pathlib.Path(src)
    return str(pathlib.Path(output_dir) / f"{p.stem}_{tag}{p.suffix.lower()}")

def file_size_bytes(p: str) -> int:
    return os.path.getsize(p)

def S_metric(C: float, VMAF: float) -> float:
    # S = 0.8 * (1 - C^1.5) + 0.2 * (VMAF - 80)/20
    return 0.8*(1 - (C**1.5)) + 0.2*((VMAF - 80.0)/20.0)

def get_source_fps_str(path: str) -> Optional[str]:
    fps = get_video_props(path)["r_frame_rate"]
    return None if (fps in (None, "0/0")) else fps

def is_hdr_like(path: str) -> bool:
    tr = get_video_props(path)["color_transfer"]
    return ("2084" in tr) or ("hlg" in tr) or ("b67" in tr)

In [16]:
# Step 3: Encode with same extension; .avi uses x264 (HEVC is invalid in AVI)

def x265_params_string(d: Dict) -> str:
    return ":".join([f"{k}={v}" for k,v in d.items()])

def encode_extaware(src: str, dst: str, crf: int,
                    maxrate: Optional[int] = None, bufsize: Optional[int] = None,
                    vf: Optional[str] = None) -> None:
    ext = pathlib.Path(dst).suffix.lower()
    cmd = ["-i", src]
    if vf:
        cmd += ["-vf", vf]

    if ext == ".avi":
        # Force x264 for AVI
        v_args = ["-c:v","libx264","-preset", X264_PRESET, "-crf", str(crf), "-pix_fmt","yuv420p"]
        a_args = ["-c:a","libmp3lame","-b:a","192k"]
        if maxrate and bufsize:
            v_args += ["-maxrate", f"{maxrate}k", "-bufsize", f"{bufsize}k"]
        cmd += v_args + a_args + [dst]
    else:
        # x265 for mp4/mkv/mov
        v_args = ["-c:v","libx265","-preset", X265_PRESET, "-crf", str(crf),
                  "-x265-params", x265_params_string(PERCEPTUAL_X265),
                  "-pix_fmt","yuv420p","-tag:v","hvc1"]
        if ext == ".mp4":
            a_args = ["-c:a","aac","-b:a","128k","-movflags","+faststart"]
        elif ext == ".mkv":
            a_args = ["-c:a","copy"]
        elif ext == ".mov":
            a_args = ["-c:a","aac","-b:a","192k","-movflags","+faststart"]
        else:
            a_args = ["-c:a","copy"]
        if maxrate and bufsize:
            v_args += ["-maxrate", f"{maxrate}k", "-bufsize", f"{bufsize}k"]
        cmd += v_args + a_args + [dst]

    code, out, err = run_cmd(cmd)
    if code != 0:
        raise RuntimeError(f"Encode failed ({ext}): {err}")

In [17]:
# Step 4: Aligned VMAF helpers

def _aligned_legs(src_for_norm: str, use_half_res: bool) -> Tuple[str, str]:
    fps = get_source_fps_str(src_for_norm)
    hdr = is_hdr_like(src_for_norm)
    chain = ["setpts=PTS-STARTPTS"]
    if hdr:
        chain += ["zscale=t=linear","tonemap=hable","zscale=matrix=bt709:transfer=bt709:primaries=bt709"]
    if fps:
        chain += [f"fps=fps={fps}"]
    if use_half_res:
        chain += ["scale=iw/2:ih/2:flags=bicubic"]
    chain += ["format=yuv420p","setsar=1"]
    return ",".join(chain)+"[ref]", ",".join(chain)+"[dist]"

def vmaf_mean_aligned_fast(ref: str,
                           dist: str,
                           src_for_norm: Optional[str] = None,
                           n_subsample: int = SAMPLING_VMAF_SUBSAMPLE,
                           half_res: bool = SAMPLING_VMAF_DOWNSCALE_HALF,
                           vmaf_threads: int = VMAF_THREADS) -> float:
    with tempfile.TemporaryDirectory() as td:
        logp = os.path.join(td, "vmaf.json")
        srcn = src_for_norm or ref
        ref_leg, dist_leg = _aligned_legs(srcn, use_half_res=half_res)
        opts = [f"n_threads={vmaf_threads}", "log_fmt=json", f"log_path='{logp}'"]
        if n_subsample and n_subsample > 1:
            opts.append(f"n_subsample={n_subsample}")
        fg = f"[0:v]{ref_leg};[1:v]{dist_leg};[dist][ref]libvmaf=" + ":".join(opts)
        code, out, err = run_cmd(["-i", ref, "-i", dist, "-map","0:v:0","-map","1:v:0", "-lavfi", fg, "-f","null","-"])
        if code != 0:
            raise RuntimeError(f"VMAF (fast) failed: {err}")
        with open(logp,"r") as f:
            data = json.load(f)
        try:
            return float(data["pooled_metrics"]["vmaf"]["mean"])
        except Exception:
            frames = data.get("frames", [])
            vals = [fr["metrics"]["vmaf"] for fr in frames if "metrics" in fr and "vmaf" in fr["metrics"]]
            if not vals: raise RuntimeError("VMAF JSON missing values")
            return sum(vals)/len(vals)

def vmaf_mean_aligned_full(ref: str,
                           dist: str,
                           src_for_norm: Optional[str] = None,
                           vmaf_threads: int = VMAF_THREADS) -> float:
    with tempfile.TemporaryDirectory() as td:
        logp = os.path.join(td, "vmaf.json")
        srcn = src_for_norm or ref
        ref_leg, dist_leg = _aligned_legs(srcn, use_half_res=False)
        fg = f"[0:v]{ref_leg};[1:v]{dist_leg};[dist][ref]libvmaf=n_threads={vmaf_threads}:log_fmt=json:log_path='{logp}'"
        code, out, err = run_cmd(["-i", ref, "-i", dist, "-map","0:v:0","-map","1:v:0", "-lavfi", fg, "-f","null","-"])
        if code != 0:
            raise RuntimeError(f"VMAF (full) failed: {err}")
        with open(logp,"r") as f:
            data = json.load(f)
        try:
            return float(data["pooled_metrics"]["vmaf"]["mean"])
        except Exception:
            frames = data.get("frames", [])
            vals = [fr["metrics"]["vmaf"] for fr in frames if "metrics" in fr and "vmaf" in fr["metrics"]]
            if not vals: raise RuntimeError("VMAF JSON missing values")
            return sum(vals)/len(vals)

In [18]:
# Step 5: Sampling C baseline aligned with final (RULE 4)

def _make_src_segment_copy(src: str, dst: str, start: float, dur: float) -> None:
    code, out, err = run_cmd([
        "-ss", str(start), "-t", str(dur), "-i", src,
        "-c", "copy",
        "-avoid_negative_ts", "make_zero",
        dst
    ])
    if code != 0:
        raise RuntimeError(f"Source segment copy failed: {err}")

def _make_ref_clip_for_vmaf(src: str, dst: str, start: float, dur: float) -> None:
    code, out, err = run_cmd([
        "-ss", str(start), "-t", str(dur), "-i", src,
        "-map","0:v:0",
        "-c:v","libx264","-preset","ultrafast","-crf","18",
        "-pix_fmt","yuv420p",
        "-an",
        dst
    ])
    if code != 0:
        raise RuntimeError(f"Ref clip creation failed: {err}")

def sample_clips(src: str, duration: int, offsets_frac: List[float]):
    props = get_video_props(src)
    offs  = [max(0.0, props["duration"]*f - duration/2) for f in offsets_frac]
    tmpd  = tempfile.TemporaryDirectory()

    src_segments = [os.path.join(tmpd.name, f"src_{i}.mp4") for i in range(len(offs))]
    ref_clips    = [os.path.join(tmpd.name, f"ref_{i}.mp4") for i in range(len(offs))]

    for i, start in enumerate(offs):
        _make_src_segment_copy(src, src_segments[i], start, duration)
        _make_ref_clip_for_vmaf(src, ref_clips[i], start, duration)

    return tmpd, src_segments, ref_clips

def evaluate_on_samples(src: str, candidates: List[Dict]) -> List[Dict]:
    tmp_ref, src_segments, ref_clips = sample_clips(src, SAMPLE_DURATION, SAMPLE_OFFSETS_FRAC)

    # Final-aligned baseline: original segments
    orig_sizes = [file_size_bytes(p) for p in src_segments]
    avg_orig   = sum(orig_sizes)/len(orig_sizes)

    # Pair each cand with each (src_segment, ref_clip)
    pairs = []
    for cand in candidates:
        for i in range(len(ref_clips)):
            pairs.append( (cand, src_segments[i], ref_clips[i]) )

    raw = []
    for cand, src_seg, ref_clip in pairs:
        with tempfile.TemporaryDirectory() as td:
            dist = os.path.join(td, f"dist_{cand['name']}.mp4")
            # encode from ref_clip (stable pix fmt for VMAF)
            encode_extaware(ref_clip, dist, crf=cand["crf"],
                            maxrate=cand["maxrate"], bufsize=cand["bufsize"], vf=None)
            q = vmaf_mean_aligned_fast(ref_clip, dist, src_for_norm=src)
            s = file_size_bytes(dist)
        raw.append( (cand["name"], cand["crf"], cand["maxrate"], cand["bufsize"], q, s) )

    # Aggregate across two clips
    agg = {}
    for name, crf, maxrate, bufsize, q, sz in raw:
        if name not in agg:
            agg[name] = {"q": [], "sz": [], "crf": crf, "maxrate": maxrate, "bufsize": bufsize}
        agg[name]["q"].append(q); agg[name]["sz"].append(sz)

    results = []
    for name, dat in agg.items():
        avg_q  = sum(dat["q"])/len(dat["q"])
        avg_sz = sum(dat["sz"])/len(dat["sz"])
        C      = min(1.0, max(0.0, avg_sz/avg_orig))
        S      = S_metric(C, avg_q)
        results.append({"name": name, "crf": dat["crf"], "maxrate": dat["maxrate"], "bufsize": dat["bufsize"],
                        "avg_vmaf_fast": avg_q, "avg_C": C, "avg_S": S})

    results.sort(key=lambda r: (r["avg_S"], r["avg_vmaf_fast"]), reverse=True)
    return results

In [19]:
# Step 6: Candidates (RULE 1 & 2)

def build_candidates(src_bitrate_bps: float,
                     enable_vbv: bool = ENABLE_VBV,
                     vbv_mults: List[float] = VBV_MAXRATE_MULTS,
                     bufsize_factor: float = VBV_BUFSIZE_FACTOR) -> List[Dict]:
    """
    By default (enable_vbv=False), build CRF-only candidates (RULE 1).
    If enable_vbv=True, add *mild* capped variants (RULE 2).
    """
    kbps = src_bitrate_bps/1000.0 if src_bitrate_bps else None
    base = [{"name": f"CRF{crf}", "crf": crf, "maxrate": None, "bufsize": None} for crf in CRF_LIST]

    if not enable_vbv or not kbps:
        return base

    caps = []
    for crf in CRF_LIST:
        for m in vbv_mults:
            mr = int(kbps * m)
            bs = int(mr * bufsize_factor)
            caps.append({"name": f"CRF{crf}_cap{int(m*100)}", "crf": crf, "maxrate": mr, "bufsize": bs})
    return base + caps


In [20]:
# Step 7: Guard + Full encode with new rescue preference (RULE 3)

def make_clip(src: str, dst: str, start: float, dur: float) -> None:
    code, out, err = run_cmd([
        "-ss", str(start), "-t", str(dur), "-i", src,
        "-map","0:v:0",
        "-c:v","libx264","-preset","ultrafast","-crf","18",
        "-pix_fmt","yuv420p",
        "-an",
        dst
    ])
    if code != 0:
        raise RuntimeError(f"Guard clip creation failed: {err}")

def verify_choice_on_midclip(src: str, chosen: Dict) -> Tuple[float, float, float, Dict]:
    props = get_video_props(src)
    start = max(0.0, props["duration"]*0.5 - SAMPLE_DURATION/2)
    with tempfile.TemporaryDirectory() as td:
        ref  = os.path.join(td, "ref_mid.mp4")
        dist = os.path.join(td, "dist_mid.mp4")
        make_clip(src, ref, start, SAMPLE_DURATION)

        adj = dict(chosen)
        for _ in range(2):
            encode_extaware(ref, dist, crf=adj["crf"], maxrate=adj["maxrate"], bufsize=adj["bufsize"], vf=None)
            V = vmaf_mean_aligned_full(ref, dist, src_for_norm=src)
            C = min(1.0, max(0.0, file_size_bytes(dist) / file_size_bytes(ref)))
            S = S_metric(C, V)
            if V >= 80.0:
                return V, C, S, adj
            # prefer removing caps first (RULE 3)
            if adj.get("maxrate") or adj.get("bufsize"):
                adj["maxrate"], adj["bufsize"] = None, None
            else:
                adj["crf"] = max(18, adj["crf"] - 2)

        V = vmaf_mean_aligned_full(ref, dist, src_for_norm=src)
        C = min(1.0, max(0.0, file_size_bytes(dist) / file_size_bytes(ref)))
        S = S_metric(C, V)
        return V, C, S, adj

def full_encode_and_report(src: str, best_result: Dict,
                           input_dir: str = INPUT_DIR,
                           output_dir: str = OUTPUT_DIR) -> Dict:
    # Guard (mid-clip)
    Vg, Cg, Sg, adj = verify_choice_on_midclip(src, best_result)
    if adj["crf"] != best_result["crf"] or adj.get("maxrate") != best_result.get("maxrate"):
        print(f"Guard adjusted → CRF {best_result['crf']} → {adj['crf']}, caps={bool(adj.get('maxrate'))}")

    # First full encode (possibly with caps if they survived guard)
    dst = out_path_for(output_dir, src, "hevc_best")
    use_pref = (best_result["avg_S"] < 0.5)
    vf = GENTLE_PREFILTER if use_pref else None

    encode_extaware(src=src, dst=dst, crf=adj["crf"], maxrate=adj["maxrate"], bufsize=adj["bufsize"], vf=vf)

    # Final metrics
    V = vmaf_mean_aligned_full(src, dst, src_for_norm=src)
    C = min(1.0, max(0.0, file_size_bytes(dst) / file_size_bytes(src)))
    S = S_metric(C, V)

    drop = best_result["avg_S"] - S

    # RULE 3 (part 1): If Final S < 0.40 → immediate re-encode WITHOUT caps (same CRF)
    if S < MIN_OK_S:
        if adj.get("maxrate") or adj.get("bufsize"):
            print(f"Final S={S:.3f} < {MIN_OK_S:.2f} → Re-encoding WITHOUT VBV caps (same CRF={adj['crf']}).")
            dst2 = out_path_for(output_dir, src, "hevc_nocap")
            encode_extaware(src=src, dst=dst2, crf=adj["crf"], maxrate=None, bufsize=None, vf=vf)
            V2 = vmaf_mean_aligned_full(src, dst2, src_for_norm=src)
            C2 = min(1.0, max(0.0, file_size_bytes(dst2) / file_size_bytes(src)))
            S2 = S_metric(C2, V2)
            if S2 > S:
                try: os.replace(dst2, dst)
                except Exception:
                    os.remove(dst); os.rename(dst2, dst)
                V, C, S = V2, C2, S2
                print(f"Switched to NO-CAP output. New metrics → VMAF={V:.2f}, C={C:.4f}, S={S:.4f}")
            else:
                try: os.remove(dst2)
                except: pass

    # RULE 3 (part 2): If drop vs sample-best is big (> MAX_S_DROP), prefer removing caps first
    elif drop > MAX_S_DROP:
        if adj.get("maxrate") or adj.get("bufsize"):
            print(f"Final S drop {drop:.3f} > {MAX_S_DROP:.2f} → Retry NO-CAP (same CRF={adj['crf']}).")
            dst2 = out_path_for(output_dir, src, "hevc_nocap")
            encode_extaware(src=src, dst=dst2, crf=adj["crf"], maxrate=None, bufsize=None, vf=vf)
            V2 = vmaf_mean_aligned_full(src, dst2, src_for_norm=src)
            C2 = min(1.0, max(0.0, file_size_bytes(dst2) / file_size_bytes(src)))
            S2 = S_metric(C2, V2)
            if S2 > S:
                try: os.replace(dst2, dst)
                except Exception:
                    os.remove(dst); os.rename(dst2, dst)
                V, C, S = V2, C2, S2
                print(f"NO-CAP improved S: {S2:.3f} (prev {S:.3f}).")
            else:
                try: os.remove(dst2)
                except: pass
        else:
            # Already no-cap; as a secondary step you may lower CRF by 2
            resc_crf = max(18, adj["crf"] - 2)
            print(f"Drop {drop:.3f} > {MAX_S_DROP:.2f} but already no-cap → Try lower CRF {adj['crf']}→{resc_crf}.")
            dst2 = out_path_for(output_dir, src, "hevc_rescue")
            encode_extaware(src=src, dst=dst2, crf=resc_crf, maxrate=None, bufsize=None, vf=vf)
            V2 = vmaf_mean_aligned_full(src, dst2, src_for_norm=src)
            C2 = min(1.0, max(0.0, file_size_bytes(dst2) / file_size_bytes(src)))
            S2 = S_metric(C2, V2)
            if S2 > S:
                try: os.replace(dst2, dst)
                except Exception:
                    os.remove(dst); os.rename(dst2, dst)
                V, C, S = V2, C2, S2
                print(f"Rescue improved S: {S2:.3f} (prev {S:.3f}).")
            else:
                try: os.remove(dst2)
                except: pass

    print("\n=== FINAL RESULTS ===")
    print("Output:", dst)
    print(f"Final VMAF: {V:.2f}")
    print(f"Final C   : {C:.4f}")
    print(f"Final S   : {S:.4f}")
    print("Targets:")
    print(" - VMAF > 80 =>", "OK" if V > 80 else "NOT MET")
    print(" - S > 0.5   =>", "OK" if S > 0.5 else "NOT MET")

    return {"output": dst, "VMAF": V, "C": C, "S": S, "prefilter_used": (best_result["avg_S"] < 0.5)}


In [21]:
def process_by_index(idx: int,
                     input_dir: str = INPUT_DIR,
                     output_dir: str = OUTPUT_DIR) -> Dict:
    check_ffmpeg()
    videos = list_input_videos(input_dir)
    if not videos:
        raise RuntimeError(f"No videos in {input_dir} with {sorted(ALLOWED_EXTS)}")

    print("Indexed videos:")
    for i, v in enumerate(videos):
        print(f"[{i}] {v}")
    if idx < 0 or idx >= len(videos):
        raise IndexError(f"Index {idx} out of range (0..{len(videos)-1})")

    src = videos[idx]
    print("\n==============================")
    print(f"[{idx}] Processing: {src}")

    props = get_video_props(src)
    candidates = build_candidates(props["bit_rate_bps"], enable_vbv=ENABLE_VBV)

    sample_results = evaluate_on_samples(src, candidates)
    best = sample_results[0]
    print("\nBest on samples (fast VMAF):")
    print(best)

    final = full_encode_and_report(src, best, input_dir, output_dir)

    print("\n=== BEST S (final) ===")
    print(f"S = {final['S']:.4f}, VMAF = {final['VMAF']:.2f}, C = {final['C']:.4f}")
    return {"samples_best": best, "final": final}

In [24]:
process_by_index(5, INPUT_DIR, OUTPUT_DIR)

FFmpeg OK: libx264/libx265/libvmaf present.
Indexed videos:
[0] /home/indranil/video_process/testbench2/VideoS10.avi
[1] /home/indranil/video_process/testbench2/VideoS11.avi
[2] /home/indranil/video_process/testbench2/VideoS12.avi
[3] /home/indranil/video_process/testbench2/VideoS13.mov
[4] /home/indranil/video_process/testbench2/VideoS14.mov
[5] /home/indranil/video_process/testbench2/VideoS16.mp4
[6] /home/indranil/video_process/testbench2/VideoS17.mp4
[7] /home/indranil/video_process/testbench2/VideoS18.mp4

[5] Processing: /home/indranil/video_process/testbench2/VideoS16.mp4

Best on samples (fast VMAF):
{'name': 'CRF34', 'crf': 34, 'maxrate': None, 'bufsize': None, 'avg_vmaf_fast': 90.5233935, 'avg_C': 0.39881112985610134, 'avg_S': 0.7037497829757134}

=== FINAL RESULTS ===
Output: /home/indranil/video_process/testbench2_output/VideoS16_hevc_best.mp4
Final VMAF: 78.89
Final C   : 0.6862
Final S   : 0.3342
Targets:
 - VMAF > 80 => NOT MET
 - S > 0.5   => NOT MET

=== BEST S (final)

{'samples_best': {'name': 'CRF34',
  'crf': 34,
  'maxrate': None,
  'bufsize': None,
  'avg_vmaf_fast': 90.5233935,
  'avg_C': 0.39881112985610134,
  'avg_S': 0.7037497829757134},
 'final': {'output': '/home/indranil/video_process/testbench2_output/VideoS16_hevc_best.mp4',
  'VMAF': 78.893507,
  'C': 0.6862002270882874,
  'S': 0.33419180760023814,
  'prefilter_used': False}}